In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from logs_parser.parser import parse_mjlog
from logs_parser.viewer import print_node
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [2]:
def relu_bn(inputs):
    relu =ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn
def residual_block(x, filters, kernel_size, downsample):
    y = Conv2D(kernel_size=kernel_size, strides= 1 if not downsample else 2, filters=filters, padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size, strides= 1, filters=filters, padding="same")(y)
    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def preprocess(df_data):
    states = []
    label = []
    for i, game in enumerate(df_data):
        node = ET.fromstringlist(game)
        data = parse_mjlog(node)
        for round in data['rounds']:
            players_hands = round[0]['data']['hands']
            encoded_hands = np.zeros((4, 4, 34))
            for j, hand in enumerate(players_hands):
                hand.sort()
                for t in hand:
                    encoded_hands[j][t % 4][t // 4] = 1

            for action in round[1:]:
                if action['tag'] not in ["DRAW", "DISCARD"]:
                    continue
                player = action['data']['player']
                same_tile_pos = action["data"]['tile'] % 4
                tile = action["data"]['tile'] // 4
                if action['tag'] == "DRAW":
                    encoded_hands[player][same_tile_pos][tile] = 1
                elif action['tag'] == "DISCARD":
                    if np.count_nonzero(encoded_hands[player]) == 14:
                        states.append(encoded_hands[player].copy())
                        label.append(tile)
                    encoded_hands[player][same_tile_pos][tile] = 0
    return states, label

In [3]:
df = pd.read_csv("./logs_parser/2021.csv")
states, label = preprocess(df['log_content'])
states = np.array(states)

In [4]:
# states = np.array_split(states, len(states)//128)
states = states.reshape((len(states), 4, 34, 1))
label = keras.utils.to_categorical(label, num_classes=34)
train = int(len(states)*0.8)
x_train = states[:train]
x_label = label[:train]
valid_data = states[train:]
valid_label = label[train:]

In [5]:
class BaseModel(keras.Model):
    def __init__(self, ):
        super(BaseModel, self).__init__()
        self.conv1 = Conv2D(kernel_size=(34, 4), strides=1, filters=34, padding="same")
        # self.residual = self.residual_block(self.conv1, 34, (34,4), True)
        self.outputs = keras.layers.Dense(34, activation="softmax")

    def residual_block(self, x, filters, kernel_size, downsample):
        y = Conv2D(kernel_size=kernel_size, strides= 1 if not downsample else 2, filters=filters, padding="same")(x)
        y = relu_bn(y)
        y = Conv2D(kernel_size=kernel_size, strides= 1, filters=filters, padding="same")(y)
        if downsample:
            x = Conv2D(kernel_size=1,
                       strides=2,
                       filters=filters,
                       padding="same")(x)
        out = Add()([x, y])
        out = relu_bn(out)
        return out
    def call(self, in_shape):
        x = in_shape
        for _ in range(3):
            x = self.conv1(x)
        x = self.residual_block(x, 34, (34,4), True)
        x = self.conv1(x)
        x = self.outputs(x)
        return keras.models.Model(in_shape,x)

In [6]:
input_shape = keras.Input((4, 34, 1))
x = input_shape
# x = x_train
x = Conv2D(256, (3, 1), padding="same", data_format="channels_last")(x)
x = Conv2D(256, (3, 1), padding="same", data_format="channels_last")(x)
x = Conv2D(256, (3, 1), padding="same", data_format="channels_last")(x)
for _ in range(50):
    x = residual_block(x, 256, (3, 1), False)

x = Conv2D(kernel_size=1, strides=1, filters=1, padding="same")(x)
x = keras.layers.Flatten()(x)

outputs = keras.layers.Dense(34, activation="softmax")(x)
# outputs = keras.backend.argmax(outputs)
model = keras.Model(input_shape,outputs)


In [ ]:
# outputs[0]
print("states shape:", states.shape)
# model.summary()
model.compile(keras.optimizers.Adam(learning_rate=0.004), keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
model.fit(x_train, x_label, epochs=50, batch_size=32, validation_data=(valid_data, valid_label), callbacks=[keras.callbacks.TensorBoard(log_dir="./logs")])


In [ ]:
# # keras.utils.plot_model(model, show_shapes=True)
# input_shape = keras.Input(( 34, 4, 1))
# model = BaseModel()(input_shape)
# # model.summary(line_length=None, positions=None, print_fn=None)
# model.compile("adam", keras.losses.categorical_crossentropy, metrics=["accuracy"])
# model.fit(states, label[:1000], epochs=50,)